In [1]:
############ LIBRARIES ###############

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import csv
import pycatch22

# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore", category=RuntimeWarning)
simplefilter(action='ignore', category=FutureWarning)

# Evaluation metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

%matplotlib inline

# variavel que evita NaN nos resultados
epslon = 0.00005


def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_pred - y_true)/np.sum(y_true))
  else:
   return 100*(np.sum(y_pred - y_true)/(np.sum(y_true)+ epslon))  

def pocid(y_true, y_pred):
  n = len(y_true)
  D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
  POCID = 100 * np.sum(D) / (n-1)
  return POCID

#função para normalização
def znorm(x):
  if np.std(x)!=0:
    x_znorm = (x - np.mean(x)) / np.std(x)
  else:
    x_znorm = (x - np.mean(x)) / (np.std(x) + epslon) 
   
  return x_znorm

#função para desnormatização
def znorm_reverse(x, mean_x, std_x):
  x_denormalized = (np.array(x) * std_x) + mean_x
  return x_denormalized

def get_stats_norm(series, horizon, window):
  last_subsequence = series[-(horizon+window):-horizon].values
  last_mean = np.mean(last_subsequence)
  last_std = np.std(last_subsequence)
  return last_mean, last_std

# Em geral, considera-se um tamanho de janela capaz de capturar um ciclo dos dados
# Por exemplo, 12 observações no caso dos dados com frequência mensal
def rolling_window(series, window):
  data = []
  for i in range(len(series)-window):
    example = znorm(np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df

# Para predição de vendas por UF (mensal), será considerado horizon = 12
# Para predição de vendas por município (anual), será considerado horizon = 1
def train_test_split(data, horizon):
  X = data.iloc[:,:-1] # features
  y = data.iloc[:,-1] # target

  X_train = X[:-horizon] # features train
  X_test =  X[-horizon:] # features test

  y_train = y[:-horizon] # target train
  y_test = y[-horizon:] # target test
  return X_train, X_test, y_train, y_test

def recursive_multistep_forecasting(X_test, model, horizon):
  # example é composto pelas últimas observações vistas
  # na prática, é o pbeprimeiro exemplo do conjunto de teste
  example = X_test.iloc[0].values.reshape(1,-1)

  preds = []
  for i in range(horizon):
    pred = model.predict(example)[0]
    preds.append(pred)

    # Descartar o valor da primeira posição do vetor de características
    example = example[:,1:]

    # Adicionar o valor predito na última posição do vetor de características
    example = np.append(example, pred)
    example = example.reshape(1,-1)
  return preds



C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
#### DEFs ####

def extract_estado(file_name):
    parts = file_name.split('_')
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    files = os.listdir(folder_path)
    for file_name in files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                headers = next(reader)
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados

#########################################################


def rolling_window_featureCatch22(series, window):
  data = []
  for i in range(len(series)-window):
      example = np.array(series[i:i+window])
      new_elements = pycatch22.catch22_all(example)
      data_feature = (new_elements['values'])
      data.append(data_feature)
  df = pd.DataFrame(data)
  return df


#########################################################


def rolling_window_featureCatch22N(series, window):
  data = []
  for i in range(len(series)-window):
      example = np.array(series[i:i+window])
      new_elements = pycatch22.catch22_all(example)
      data_feature = znorm(new_elements['values'])
      data.append(data_feature)
  df = pd.DataFrame(data)
  return df


#########################################################


def recursive_multistep_forecasting_Catch22N(series, model, horizon, window):
    serieatu=series[:-horizon]
    seriec = serieatu.tail(window)
    mean_norm, std_norm = get_stats_norm(series, horizon, window)
    predsreal = []
    for i in range(horizon):
        example = np.array(seriec)
        new_elements = pycatch22.catch22_all(example)
        exemple_feature = znorm(new_elements['values'])
        exemple_feature_df = pd.DataFrame(exemple_feature)
        exemple_feature_df.fillna(0, inplace=True)
        predn = model.predict(exemple_feature_df.T)[0]
        pred = znorm_reverse(predn, mean_norm, std_norm)
        predsreal.append(pred)
        series2 = seriec[1:]
        seriec = np.append(series2, pred)
    return predsreal


In [4]:
############## Catch22 SAVE ##############

import pickle

horizon = 12
window = 12
features = 'Catch22'

## Depende da janela e da Feature
if window == 12:
    FeatRegister = 398 
elif window ==24:
    FeatRegister = 386
else:
    FeatRegister = 374

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    
    for estado in estados:

        # carregamento do arquivo

        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
        series = df['m3']

        TimeStamp = df['timestamp'].tail(FeatRegister)
        TimeStamp.reset_index(drop=True, inplace=True)

        Catch22 = rolling_window_featureCatch22(series, window)
        Catch22_filled = Catch22.fillna(0)

        # first_column = Catch22_filled.pop('timestamp')
        Catch22_filled.insert(0, 'timestamp', TimeStamp)

        folder_name = 'Catch22'
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        
        Catch22_filled.to_csv(f'{folder_name}/FEAT_Catch22_{product}_{window}_{estado}.csv', index=False)


KeyboardInterrupt: 

In [7]:
############## Catch22 TEST_NORM ##############

import pickle

horizon = 12
window = 12
features = 'Catch22'

## Depende da janela e da Feature
if window == 12:
    FeatRegister = 398 
elif window ==24:
    FeatRegister = 386
else:
    FeatRegister = 374

#################################################
product = 'etanolhidratado'
estado = 'ac'
#################################################

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
# for product in products:
#     folder_path = f'./uf/{product}/'
#     # Read the CSV files and extract estado names
#     estados = read_csv_files(folder_path)
#     for estado in estados:

# carregamento do arquivo

df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
series = df['m3']

Catch22 = rolling_window_featureCatch22N(series, window)

Catch22_filled = Catch22.fillna(0)
Catch22_filled.replace([float('inf'), -float('inf')], 0, inplace=True)

y_norm = rolling_window(series,window)

Catch22_filled['y'] = y_norm.iloc[:, [-1]]

X_train, X_test, y_train, y_test = train_test_split(Catch22_filled, horizon)

############## Regressores ##################

regr1 = LinearRegression()
regr2 = KNeighborsRegressor(n_neighbors = 3)
regr3 = XGBRegressor()
regr4 = SVR(kernel='rbf')
regr5 = RandomForestRegressor()
regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)


regr1.fit(X_train, y_train)
predictions1a = recursive_multistep_forecasting_Catch22N(series, regr1, horizon, window)

regr2.fit(X_train, y_train)
predictions2a = recursive_multistep_forecasting_Catch22N(series, regr2, horizon, window)

regr3.fit(X_train, y_train)
predictions3a = recursive_multistep_forecasting_Catch22N(series, regr3, horizon, window)

regr4.fit(X_train, y_train)
predictions4a = recursive_multistep_forecasting_Catch22N(series, regr4, horizon, window)

regr5.fit(X_train, y_train)
predictions5a = recursive_multistep_forecasting_Catch22N(series, regr5, horizon, window)

regr6.fit(X_train, y_train)
predictions6a = recursive_multistep_forecasting_Catch22N(series, regr6, horizon, window)

Valores_Reais = series.tail(12)
Valores_Reais = Valores_Reais.reset_index(drop=True)

mape_result1 = mape(Valores_Reais, predictions1a)
mape_result2 = mape(Valores_Reais, predictions2a)
mape_result3 = mape(Valores_Reais, predictions3a)
mape_result4 = mape(Valores_Reais, predictions4a)
mape_result5 = mape(Valores_Reais, predictions5a)
mape_result6 = mape(Valores_Reais, predictions6a)

pbe_result1 = pbe(Valores_Reais, predictions1a)
pbe_result2 = pbe(Valores_Reais, predictions2a)
pbe_result3 = pbe(Valores_Reais, predictions3a)
pbe_result4 = pbe(Valores_Reais, predictions4a)
pbe_result5 = pbe(Valores_Reais, predictions5a)
pbe_result6 = pbe(Valores_Reais, predictions6a)

pocid_result1 = pocid(Valores_Reais, predictions1a)
pocid_result2 = pocid(Valores_Reais, predictions2a)
pocid_result3 = pocid(Valores_Reais, predictions3a)
pocid_result4 = pocid(Valores_Reais, predictions4a)
pocid_result5 = pocid(Valores_Reais, predictions5a)
pocid_result6 = pocid(Valores_Reais, predictions6a)

predictions1b = ', '.join(f"{item:f}" for item in predictions1a)
predictions2b = ', '.join(f"{item:f}" for item in predictions2a)
predictions3b = ', '.join(f"{item:f}" for item in predictions3a)
predictions4b = ', '.join(f"{item:f}" for item in predictions4a)
predictions5b = ', '.join(f"{item:f}" for item in predictions5a)
predictions6b = ', '.join(f"{item:f}" for item in predictions6a)

rows_data = [
    [product,estado,'LR',mape_result1,pocid_result1,pbe_result1,predictions1b],
    [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,predictions2b],
    [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,predictions3b],
    [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,predictions4b],
    [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,predictions5b],
    [product,estado,'MLP',mape_result6,pocid_result6,pbe_result6,predictions6b],
]


# # Salva Modelos
# folder_path = f"./00-MODELS_UF_MENSAL-Catch22-Norm/"
# os.makedirs(folder_path, exist_ok=True)

# with open(os.path.join(folder_path, f'Catch22_LR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd1:
#     pickle.dump(regr2, fd1)
# with open(os.path.join(folder_path, f'Catch22_KNN_{product}_{estado}_{window}_model.pkl'), 'wb') as fd2:
#     pickle.dump(regr2, fd2)
# with open(os.path.join(folder_path, f'Catch22_XGB_{product}_{estado}_{window}_model.pkl'), 'wb') as fd3:
#     pickle.dump(regr3, fd3)
# with open(os.path.join(folder_path, f'Catch22_SVR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd4:
#     pickle.dump(regr4, fd4)
# with open(os.path.join(folder_path, f'Catch22_RF_{product}_{estado}_{window}_model.pkl'), 'wb') as fd5:
#     pickle.dump(regr5, fd5)
# with open(os.path.join(folder_path, f'Catch22_MLP_{product}_{estado}_{window}_model.pkl'), 'wb') as fd6:
#     pickle.dump(regr2, fd6)

# CSV Output VALORES REAIS
with open(f'Catch22_{window}_Norm_output.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    for row_data in rows_data:
        writer.writerow(row_data)

C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [4]:
############## Catch22 AUTO_NORM ##############

import pickle

horizon = 12
window = 12
features = 'Catch22'

## Depende da janela e da Feature
if window == 12:
    FeatRegister = 398 
elif window ==24:
    FeatRegister = 386
else:
    FeatRegister = 374

#################################################
# product = 'etanolhidratado'
# estado = 'ac'
#################################################

products = sorted([name for name in os.listdir('./uf/') if os.path.isdir(os.path.join('./uf/', name))])
 
for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    for estado in estados:

        # carregamento do arquivo

        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
        series = df['m3']

        Catch22 = rolling_window_featureCatch22N(series, window)

        Catch22_filled = Catch22.fillna(0)
        Catch22_filled.replace([float('inf'), -float('inf')], 0, inplace=True)

        y_norm = rolling_window(series,window)

        Catch22_filled['y'] = y_norm.iloc[:, [-1]]

        X_train, X_test, y_train, y_test = train_test_split(Catch22_filled, horizon)

        ############## Regressores ##################

        regr1 = LinearRegression()
        regr2 = KNeighborsRegressor(n_neighbors = 3)
        regr3 = XGBRegressor()
        regr4 = SVR(kernel='rbf')
        regr5 = RandomForestRegressor()
        regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)


        regr1.fit(X_train, y_train)
        predictions1a = recursive_multistep_forecasting_Catch22N(series, regr1, horizon, window)

        regr2.fit(X_train, y_train)
        predictions2a = recursive_multistep_forecasting_Catch22N(series, regr2, horizon, window)

        regr3.fit(X_train, y_train)
        predictions3a = recursive_multistep_forecasting_Catch22N(series, regr3, horizon, window)

        regr4.fit(X_train, y_train)
        predictions4a = recursive_multistep_forecasting_Catch22N(series, regr4, horizon, window)

        regr5.fit(X_train, y_train)
        predictions5a = recursive_multistep_forecasting_Catch22N(series, regr5, horizon, window)

        regr6.fit(X_train, y_train)
        predictions6a = recursive_multistep_forecasting_Catch22N(series, regr6, horizon, window)

        Valores_Reais = series.tail(12)
        Valores_Reais = Valores_Reais.reset_index(drop=True)

        mape_result1 = mape(Valores_Reais, predictions1a)
        mape_result2 = mape(Valores_Reais, predictions2a)
        mape_result3 = mape(Valores_Reais, predictions3a)
        mape_result4 = mape(Valores_Reais, predictions4a)
        mape_result5 = mape(Valores_Reais, predictions5a)
        mape_result6 = mape(Valores_Reais, predictions6a)

        pbe_result1 = pbe(Valores_Reais, predictions1a)
        pbe_result2 = pbe(Valores_Reais, predictions2a)
        pbe_result3 = pbe(Valores_Reais, predictions3a)
        pbe_result4 = pbe(Valores_Reais, predictions4a)
        pbe_result5 = pbe(Valores_Reais, predictions5a)
        pbe_result6 = pbe(Valores_Reais, predictions6a)

        pocid_result1 = pocid(Valores_Reais, predictions1a)
        pocid_result2 = pocid(Valores_Reais, predictions2a)
        pocid_result3 = pocid(Valores_Reais, predictions3a)
        pocid_result4 = pocid(Valores_Reais, predictions4a)
        pocid_result5 = pocid(Valores_Reais, predictions5a)
        pocid_result6 = pocid(Valores_Reais, predictions6a)

        predictions1b = ', '.join(f"{item:f}" for item in predictions1a)
        predictions2b = ', '.join(f"{item:f}" for item in predictions2a)
        predictions3b = ', '.join(f"{item:f}" for item in predictions3a)
        predictions4b = ', '.join(f"{item:f}" for item in predictions4a)
        predictions5b = ', '.join(f"{item:f}" for item in predictions5a)
        predictions6b = ', '.join(f"{item:f}" for item in predictions6a)

        rows_data = [
            [product,estado,'LR',mape_result1,pocid_result1,pbe_result1,predictions1b],
            [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,predictions2b],
            [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,predictions3b],
            [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,predictions4b],
            [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,predictions5b],
            [product,estado,'MLP',mape_result6,pocid_result6,pbe_result6,predictions6b],
        ]

        
        # CSV Output VALORES REAIS
        with open(f'Catch22_{window}_Norm_output.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            for row_data in rows_data:
                writer.writerow(row_data)


        # # Salva Modelos
        # folder_path = f"./00-MODELS_UF_MENSAL-Catch22-Norm/"
        # os.makedirs(folder_path, exist_ok=True)

        # with open(os.path.join(folder_path, f'Catch22_LR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd1:
        #     pickle.dump(regr2, fd1)
        # with open(os.path.join(folder_path, f'Catch22_KNN_{product}_{estado}_{window}_model.pkl'), 'wb') as fd2:
        #     pickle.dump(regr2, fd2)
        # with open(os.path.join(folder_path, f'Catch22_XGB_{product}_{estado}_{window}_model.pkl'), 'wb') as fd3:
        #     pickle.dump(regr3, fd3)
        # with open(os.path.join(folder_path, f'Catch22_SVR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd4:
        #     pickle.dump(regr4, fd4)
        # with open(os.path.join(folder_path, f'Catch22_RF_{product}_{estado}_{window}_model.pkl'), 'wb') as fd5:
        #     pickle.dump(regr5, fd5)
        # with open(os.path.join(folder_path, f'Catch22_MLP_{product}_{estado}_{window}_model.pkl'), 'wb') as fd6:
        #     pickle.dump(regr2, fd6)


C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jonas\A